# Feature Store as a Hub for ML

## Setup Project

Create project to separate resources.

In [1]:
import pandas as pd
import mlrun
import mlrun.feature_store as fstore
from mlrun.datastore.sources import DataFrameSource, CSVSource, ParquetSource

In [2]:
project = mlrun.get_or_create_project(name="berkeley-mlops", context=".")

> 2022-09-06 17:35:06,189 [info] loaded project berkeley-mlops from MLRun DB


---

## Ingest

### What data are we using?

We will be using a simple heart disease dataset for ingestion and retrieval. The dataset itself will be separated across 3 different files to simulate multiple data sources. However, there is a single `patiend_id` column that is the same across all datasets and will be used to join them together.

---

### Define Feature Sets

We will need to define a `Feature Set` per data source. This will look something like the following:

In [3]:
categorical_fset = fstore.FeatureSet(
    name="heart-disease-categorical",
    entities=[fstore.Entity("patient_id")],
    description="Categorical columns for heart disease dataset"
)

In [4]:
continuous_fset = fstore.FeatureSet(
    name="heart-disease-continuous",
    entities=[fstore.Entity("patient_id")],
    description="Continuous columns for heart disease dataset"
)

In [5]:
target_fset = fstore.FeatureSet(
    name="heart-disease-target",
    entities=[fstore.Entity("patient_id")],
    description="Target column for heart disease dataset"
)

---

### Ingest Data into Feature Sets

Now that our `Feature Sets` are defined, we can ingest data into them as follows. We will be using the `DataFrameSource`, `CSVSource`, and `ParquetSource` for this example, although you could use just one.

#### Ingest Categorical data using DataFrameSource

In [6]:
categorical_df = pd.read_csv("data/heart_disease_categorical.csv")

In [7]:
fstore.ingest(
    featureset=categorical_fset,
    source=DataFrameSource(df=categorical_df)  # Use DataFrameSource and pandas dataframe from above
)

Converting input from bool to <class 'numpy.uint8'> for compatibility.


,age,sex,cp,exang,fbs,slope,thal
patient_id,,,,,,,
e443544b-8d9e-4f6c-9623-e24b6139aae0,52,male,typical_angina,no,False,downsloping,normal
8227d3df-16ab-4452-8ea5-99472362d982,53,male,typical_angina,yes,True,upsloping,normal
10c4b4ba-ab40-44de-8aba-6bdb062192c4,70,male,typical_angina,yes,False,upsloping,normal
f0acdc22-7ee6-4817-a671-e136211bc0a6,61,male,typical_angina,no,False,downsloping,normal
2d6b3bca-4841-4618-9a8c-ca902010b009,62,female,typical_angina,no,True,flat,reversable_defect
...,...,...,...,...,...,...,...
5d2fc80f-ed64-4e1c-9c95-3baace09118b,59,male,atypical_angina,yes,False,downsloping,reversable_defect
01548a7e-0f68-4308-80de-cd93fdbfb903,60,male,typical_angina,yes,False,flat,normal
f8c97cc1-8a3a-4b8e-965c-58e75c2379e6,47,male,typical_angina,yes,False,flat,reversable_defect


#### Ingest Continuous data using CSVSource

In [8]:
fstore.ingest(
    featureset=continuous_fset,
    source=CSVSource(path="./data/heart_disease_continuous.csv")  # Use CSVSource and path to CSV file
)

,trestbps,chol,restecg,thalach,oldpeak,ca
patient_id,,,,,,
e443544b-8d9e-4f6c-9623-e24b6139aae0,125,212,1,168,1.0,2.0
8227d3df-16ab-4452-8ea5-99472362d982,140,203,0,155,3.1,0.0
10c4b4ba-ab40-44de-8aba-6bdb062192c4,145,174,1,125,2.6,0.0
f0acdc22-7ee6-4817-a671-e136211bc0a6,148,203,1,161,0.0,1.0
2d6b3bca-4841-4618-9a8c-ca902010b009,138,294,1,106,1.9,3.0
...,...,...,...,...,...,...
5d2fc80f-ed64-4e1c-9c95-3baace09118b,140,221,1,164,0.0,0.0
01548a7e-0f68-4308-80de-cd93fdbfb903,125,258,0,141,2.8,1.0
f8c97cc1-8a3a-4b8e-965c-58e75c2379e6,110,275,0,118,1.0,1.0


#### Ingest Target data using ParquetSource

In [9]:
fstore.ingest(
    featureset=target_fset,
    source=ParquetSource(path="./data/heart_disease_target.parquet")  # Use ParquetSource and path to parquet file
)

,target
patient_id,
e443544b-8d9e-4f6c-9623-e24b6139aae0,0
8227d3df-16ab-4452-8ea5-99472362d982,0
10c4b4ba-ab40-44de-8aba-6bdb062192c4,0
f0acdc22-7ee6-4817-a671-e136211bc0a6,0
2d6b3bca-4841-4618-9a8c-ca902010b009,0
...,...
5d2fc80f-ed64-4e1c-9c95-3baace09118b,1
01548a7e-0f68-4308-80de-cd93fdbfb903,0
f8c97cc1-8a3a-4b8e-965c-58e75c2379e6,0


---

## Retrieve

### Define Feature Vector

Our `Feature Vector` will allow us to join and retrieve the 3 previous `Feature Sets` together as a single dataset. We can define it like so:

In [10]:
fvec = fstore.FeatureVector(
    name="heart-disease-vector",
    features=["heart-disease-categorical.*", "heart-disease-continuous.*"],
    description="Heart disease dataset",
    label_feature="heart-disease-target.target"
)
fvec.save()

### Retrieve Feature Vector as Dataframe (Training)

In order to retrieve our `Feature Vector`, we just need to specify the project and name of the vector.

One of the most common ways to retrieve your dataset is as a dataframe. This is very simple to do:

In [11]:
df = fstore.get_offline_features("heart-disease-vector").to_dataframe()
df.head()

,age,sex,cp,exang,fbs,slope,thal,trestbps,chol,restecg,thalach,oldpeak,ca,target
0,52,male,typical_angina,no,False,downsloping,normal,125,212,1,168,1.0,2.0,0
1,53,male,typical_angina,yes,True,upsloping,normal,140,203,0,155,3.1,0.0,0
2,70,male,typical_angina,yes,False,upsloping,normal,145,174,1,125,2.6,0.0,0
3,61,male,typical_angina,no,False,downsloping,normal,148,203,1,161,0.0,1.0,0
4,62,female,typical_angina,no,True,flat,reversable_defect,138,294,1,106,1.9,3.0,0


### Retrieve Records in Real-Time (Serving)

In addition to retrieving the entire dataset in batch, you can also retrieve individual records in real-time using their `Entity`. This is useful for enriching incoming records and imputing missing values.

To retrieve records in real-time, we can create an online feature service like so:

In [12]:
feature_service = fstore.get_online_feature_service(feature_vector="heart-disease-vector")

Then, we can retrieve a record like so:

In [13]:
feature_service.get(
    [
        {"patient_id" : "e443544b-8d9e-4f6c-9623-e24b6139aae0"},
        {"patient_id" : "8227d3df-16ab-4452-8ea5-99472362d982"}
    ]
)

[{'age': 52,
  'sex': 'male',
  'cp': 'typical_angina',
  'exang': 'no',
  'fbs': False,
  'slope': 'downsloping',
  'thal': 'normal',
  'trestbps': 125,
  'chol': 212,
  'restecg': 1,
  'thalach': 168,
  'oldpeak': 1.0,
  'ca': 2.0},
 {'age': 53,
  'sex': 'male',
  'cp': 'typical_angina',
  'exang': 'yes',
  'fbs': True,
  'slope': 'upsloping',
  'thal': 'normal',
  'trestbps': 140,
  'chol': 203,
  'restecg': 0,
  'thalach': 155,
  'oldpeak': 3.1,
  'ca': 0.0}]